In [1]:
# import reddit api wrapper
import praw
import pandas as pd
import numpy as np
import datetime
import sqlite3

from textblob import TextBlob

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're one level up
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/Georgetown/Spring2023/TimeSeries/ANLY560_website


In [10]:
%autoreload
from _functions.get_reddit_data import get_submissions, get_comments

In [ ]:
with sqlite3.connect("_data/magic_reddit.db") as conn:

    submissions = pd.read_sql_query("SELECT submission_id FROM submissions", conn)
    print("Total Submissions: " + str(len(submissions)))

    # pull in all submission ids that have previously been crawled
    df = pd.read_sql_query("SELECT * FROM comments", conn)
    crawled_ids = df['submission_id'].unique().tolist()
    print("Crawled Submissions: " + str(len(crawled_ids)))

    # yields the elements in all_ids not in crawled_ids
    remaining_ids = np.setdiff1d(submissions.submission_id, crawled_ids)
    print("Submissions remaining: " + str(len(remaining_ids)))

    for each_id in remaining_ids:

        comment_df = get_comments(each_id)
        
        comment_df.to_sql(name='comments', con=conn, if_exists='append', index=False)